# Screened Infeasibility UQ: evaluate only

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the response/quantity of interest as the violated constraint residuals, i.e. the infeasibility. This is inexpensive enough to run locally.

Here, the inputs have already been screened using the single-parameter evaluation method to find the most sensitive inputs. This notebook just evaluates the samples and stores them in a database, ready for analysing in in the `infeas_screened_analysis.ipynb` notebook as a later stage.

In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
import numpy as np
from typing import Dict, Any, Optional
from process.io.mfile import MFile
import regex as re

## Campaign to capture infeasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals. To start with, make just 3 inputs uncertain (for running locally).

Begin by defining the parameter bounds and the responses (or quantities of interest).

In [2]:
# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_local", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex's SA paper

params = {
    "fdene": {
        "type": "float",
        "min": 1.1,
        "max": 1.3,
        "default": 1.2,
    },  # check: not sure if this is right var. I think ok
    "hfact": {"type": "float", "min": 1.0, "max": 1.2, "default": 1.2},
    "coreradius": {"type": "float", "min": 0.45, "max": 0.75, "default": 0.75},  # ok
    "fimp_2": {"type": "float", "min": 0.085, "max": 0.115, "default": 0.1},  # ok
    "fimp_14": {
        "type": "float",
        "min": 1.0e-5,
        "max": 1.0e-4,
        "default": 1.0e-5,
    },  # ok
    "psepbqarmax": {"type": "float", "min": 8.7, "max": 9.7, "default": 9.0},  # ok
    "flhthresh": {"type": "float", "min": 0.85, "max": 1.15, "default": 1.15},  # ok
    "cboot": {
        "type": "float",
        "min": 0.95,
        "max": 1.05,
        "default": 1.0,
    },  # ok
    "peakfactrad": {"type": "float", "min": 2.0, "max": 3.5, "default": 3.33},  # ok
    "kappa": {"type": "float", "min": 1.8, "max": 1.9, "default": 1.848},  # ok
    "etaech": {"type": "float", "min": 0.3, "max": 0.5, "default": 0.4},  # ok
    "feffcd": {"type": "float", "min": 0.5, "max": 5.0, "default": 1.0},  # ok
    "etath": {"type": "float", "min": 0.36, "max": 0.4, "default": 0.375},  # ok
    "etaiso": {"type": "float", "min": 0.75, "max": 0.95, "default": 0.9},  # ok
    "boundl_18": {
        "type": "float",
        "min": 3.25,
        "max": 3.75,
        "default": 3.25,
    },  # q^95_min, ok
    "pinjalw": {"type": "float", "min": 51.0, "max": 61.0, "default": 61.0},  # ok
    "alstroh": {"type": "float", "min": 6.0e8, "max": 7.2e8, "default": 6.6e8},  # ok
    "sig_tf_wp_max": {
        "type": "float",
        "min": 5.2e8,
        "max": 6.4e8,
        "default": 6.4e8,
    },  # ok, but might need sig_tf_case_max to be the same too
    "aspect": {"type": "float", "min": 3.0, "max": 3.2, "default": 3.1},
    "boundu_2": {
        "type": "float",
        "min": 11.0,
        "max": 12.0,
        "default": 12.0,
    },  # B_T^max, ok
    "triang": {"type": "float", "min": 0.4, "max": 0.6, "default": 0.5},  # ok
    "vary_param": {
        "type": "string",
        "default": "",
    },  # param being changed: used for analysis only
}

# QoIs
# Violated constraint residuals
qois = [
    "norm_objf",
    "eq_con001",
    "eq_con002",
    "eq_con011",
    "ineq_con005",
    "ineq_con008",
    "ineq_con009",
    "ineq_con013",
    "ineq_con015",
    "ineq_con030",
    "ineq_con016",
    "ineq_con024",
    "ineq_con025",
    "ineq_con026",
    "ineq_con027",
    "ineq_con033",
    "ineq_con034",
    "ineq_con035",
    "ineq_con036",
    "ineq_con060",
    "ineq_con062",
    "ineq_con065",
    "ineq_con072",
    "ineq_con079",
    "ineq_con081",
    "ineq_con068",
    "ineq_con031",
    "ineq_con032",
    "rms_vio_constr_res",
]

Next, define a custom decoder to handle Process's mfile output.

In [3]:
class MfileDecoder(uq.decoders.JSONDecoder):
    """Interprets Process's mfiles to extract repsonses for easyVVUQ.

    Subclasses easyVVUQ's JSON decoder.
    """

    def _get_raw_data(self, out_path: str) -> Dict:
        """Parse mfile and return dictionary of all output data.

        Parameters
        ----------
        out_path : str
            Path to mfile

        Returns
        -------
        Dict
            All output data contained in mfile
        """
        mfile = MFile(Path(out_path))
        mfile_dict = {}
        for param_name in mfile.data:
            param_value = mfile.data[param_name].get_scan(-1)
            mfile_dict[param_name] = param_value

        return mfile_dict

    def _process_raw_data(self, raw_data: Dict[str, Any]) -> Dict[str, float]:
        """Perform any required processing of raw mfile data dict.

        May include filtering for desired responses.

        Parameters
        ----------
        raw_data : Dict[str, Any]
            Entire raw output dictionary

        Returns
        -------
        Dict[str, float]
            Processed output dictionary
        """
        # Add objective function to responses dict
        responses = {"norm_objf": raw_data["norm_objf"]}

        # Extract constraints from raw_data dict
        # Find all equality and inequality constraint keys
        eq_re = re.compile(r"eq_con\d{3}")
        ineq_re = re.compile(r"ineq_con\d{3}")
        eq_constrs_keys = [key for key in raw_data.keys() if eq_re.match(key)]
        ineq_constrs_keys = [key for key in raw_data.keys() if ineq_re.match(key)]

        # Get values of constraints
        eq_constrs_dict = {
            eq_constrs_key: raw_data[eq_constrs_key]
            for eq_constrs_key in eq_constrs_keys
        }
        ineq_constrs_dict = {
            ineq_constrs_key: raw_data[ineq_constrs_key]
            for ineq_constrs_key in ineq_constrs_keys
        }

        # Only want violated constraint values
        # Coerce feasible inequality constraints (> 0) = 0.0
        # TODO Not sure if we want to mask non-violated constraint
        # values at this stage: infeasibile responses only
        vio_ineq_constrs_dict = {}
        for key, value in ineq_constrs_dict.items():
            if value > 0:
                vio_ineq_constrs_dict[key] = 0.0
            else:
                vio_ineq_constrs_dict[key] = value

        # Merge individual eq and ineq violated constraint values
        responses = responses | eq_constrs_dict | vio_ineq_constrs_dict

        # Calculate RMS constraint residuals for violated constraints only
        # Create arrays from constraints dicts
        eq_constrs = np.array(list(eq_constrs_dict.values()))
        vio_ineq_constrs = np.array(list(vio_ineq_constrs_dict.values()))
        vio_constrs = np.concatenate((eq_constrs, vio_ineq_constrs))
        rms_vio_constr_res = np.sqrt(np.mean(vio_constrs**2))
        responses["rms_vio_constr_res"] = rms_vio_constr_res

        return responses

    def parse_sim_output(self, run_info: Optional[Dict] = None) -> Dict[str, float]:
        """Parse mfile, process it and return dict to easyVVUQ.

        Adapted from JSON decoder source to include _process_raw_data() step.

        Parameters
        ----------
        run_info : Optional[Dict], optional
            Run information supplied by easyVVUQ, by default None

        Returns
        -------
        Dict[str, float]
            Response data for easyVVUQ

        Raises
        ------
        RuntimeError
            Raised if field is absent from processed output data
        """
        if run_info is None:
            run_info = {}

        def get_value(data, path):
            for node in path:
                data = data[node]
            return data

        out_path = self._get_output_path(run_info, self.target_filename)
        raw_data = self._get_raw_data(out_path)

        # Perform any required processing of raw data
        processed_data = self._process_raw_data(raw_data)

        data = []
        for col in self.output_columns:
            try:
                if isinstance(col, str):
                    data.append((col, processed_data[col]))
                elif isinstance(col, list):
                    data.append((".".join(col), get_value(processed_data, col)))
            except KeyError:
                raise RuntimeError("no such field: {} in this mfile".format(col))
        return dict(data)

In [4]:
# Create encoder and decoder
encoder = uq.encoders.GenericEncoder(
    template_fname="demo_sol_max_net_elec_no_f_IN.template", target_filename="IN.DAT"
)

decoder = MfileDecoder(target_filename="MFILE.DAT", output_columns=qois)

cmd = "process -i IN.DAT"
actions = uq.actions.local_execute(encoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

In [5]:
# Create PCE sampler
# Vary 3 uncertain inputs
vary = {
    # "fdene": cp.Uniform(
    #     1.1,
    #     1.3,
    # ),
    # "hfact": cp.Uniform(
    #     1.0,
    #     1.2,
    # ),
    # "coreradius": cp.Uniform(
    #     0.45,
    #     0.75,
    # ),
    # "fimp_2": cp.Uniform(
    #     0.085,
    #     0.115,
    # ),
    "fimp_14": cp.Uniform(
        1.0e-5,
        1.0e-4,
    ),
    "psepbqarmax": cp.Uniform(
        8.7,
        9.7,
    ),
    # "flhthresh": cp.Uniform(
    #     0.85,
    #     1.15,
    # ),
    # "cboot": cp.Uniform(
    #     0.95,
    #     1.05,
    # ),
    # "peakfactrad": cp.Uniform(
    #     2.0,
    #     3.5,
    # ),
    # "kappa": cp.Uniform(
    #     1.8,
    #     1.9,
    # ),
    # "etaech": cp.Uniform(
    #     0.3,
    #     0.5,
    # ),
    # "feffcd": cp.Uniform(
    #     0.5,
    #     5.0,
    # ),
    # "etath": cp.Uniform(
    #     0.36,
    #     0.4,
    # ),
    # "etaiso": cp.Uniform(
    #     0.75,
    #     0.95,
    # ),
    # "boundl_18": cp.Uniform(
    #     3.25,
    #     3.75,
    # ),
    "pinjalw": cp.Uniform(
        51.0,
        61.0,
    ),
    # "alstroh": cp.Uniform(
    #     6.0e8,
    #     7.2e8,
    # ),
    # "sig_tf_wp_max": cp.Uniform(
    #     5.2e8,
    #     6.4e8,
    # ),
    # "aspect": cp.Uniform(
    #     3.0,
    #     3.2,
    # ),
    # "boundu_2": cp.Uniform(
    #     11.0,
    #     12.0,
    # ),
    # "triang": cp.Uniform(
    #     0.4,
    #     0.6,
    # ),
}
pce_sampler = uq.sampling.PCESampler(vary=vary, polynomial_order=3)

# Add pce_sampler to campaign
campaign.set_sampler(pce_sampler)

# Draw samples, execute and collate
campaign.execute().collate(progress_bar=True)
samples = campaign.get_collation_result()

  0%|          | 0/64 [00:00<?, ?it/s]

The IN.DAT file does not contain any obsolete variables.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. :  code contains untracked changes
    Branch : constraint-info-for-uq
   Git log : Add actual constraint numbers to mfile
 Date/time :  2 Feb 2024 17:05:43 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-356

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
  2%|▏         | 1/64 [00:02<02:07,  2.03s/it]

The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. :  code contains untracked changes
    Branch : constraint-info-f

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.


...

 94%|█████████▍| 60/64 [00:24<00:01,  3.39it/s]

 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. :  code contains untracked changes
    Branch : constraint-info-for-uq
   Git log : Add actual constraint numbers to mfile
 Date/time :  2 Feb 2024 17:06:06 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory :
 Warning in routine OCMMNT :
    Input : /

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
 98%|█████████▊| 63/64 [00:25<00:00,  4.98it/s]

 triang: 0.4 - 0.6; kappa: 1.5 - 2.0;   plascur: 10 - 20 MA, rmajor: 7 - 11 m;
 rminor: 2 - 3.5 m; tesep: 0 - 0.5 keV; normalised_total_beta: 2 - 3
   normalised_total_beta =    3.2554E+00
 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  
 
 ID  LEVEL  MESSAGE
243     2   PHYSICS: Predicted plasma driven current is more than upper limit on non-inducti
  
244     2   PHYSICS: Diamagnetic fraction is more than 1%, but not calculated. Consider usin
  
135     1   OUTPF: CS not using max current density: further optimisation may be possible   
  
 
 ******************************************* End of PROCESS Output ********************************************
 


100%|██████████| 64/64 [00:25<00:00,  2.53it/s]
